In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import pandas as pd

# Load data
data = 'data_set.csv'
df = pd.read_csv(data, header=0)
X = df.drop(['ID_REF', '!Sample_source_name_ch1', 'Accession', 'Tissue Group', 'Tissue Group ID', 'Gender', 'Individual', 'Tissue', 'Disease state'], axis=1)
y = df['Disease state']

# Impute missing values
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)
X = pd.DataFrame(X_imputed, columns=X.columns)  # Convert back to DataFrame to keep column names

# Data balancing with SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Feature selection using RFECV
model = RandomForestClassifier(random_state=42)
rfecv = RFECV(estimator=model, step=1, cv=StratifiedKFold(5), scoring='accuracy')
X_train_selected = rfecv.fit_transform(X_train, y_train)
X_test_selected = rfecv.transform(X_test)

# Feature Importances and Hub Genes
model.fit(X_train_selected, y_train)  # Ensure the model is refit for feature importance extraction
feature_importances = pd.Series(model.feature_importances_, index=X.columns[rfecv.support_]).sort_values(ascending=False)

# Ensemble model
lr = LogisticRegression(max_iter=1000)
svm = SVC(probability=True)
ensemble = VotingClassifier(estimators=[('lr', lr), ('rf', model), ('svm', svm)], voting='soft')
ensemble.fit(X_train_selected, y_train)

# Predict
y_pred = ensemble.predict(X_test_selected)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print results
print(f'Model Accuracy: {accuracy:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')
print(f'F1 Score: {f1:.4f}')
print('Top 10 Hub Genes by Feature Importance:')
print(feature_importances.head(10))


Model Accuracy: 0.8182
Recall: 0.8182
Precision: 0.8179
F1 Score: 0.8171
Top 10 Hub Genes by Feature Importance:
Age             0.045923
ILMN_1651799    0.023760
ILMN_1651254    0.018072
ILMN_1651557    0.016786
ILMN_1651569    0.012888
ILMN_1651699    0.012462
ILMN_1652128    0.011409
ILMN_1652412    0.011405
ILMN_1651715    0.010438
ILMN_1652368    0.009901
dtype: float64
